# Linked Births and Deaths

The CDC provides a linking between births and infant deaths here: https://www.cdc.gov/nchs/data_access/vitalstatsonline.htm#Downloadable 

These data have very similar columns as the natality set, so it's important for understanding the distributions of infant fatalities based on other factors. There are exploratory publications (e.g., https://www.cdc.gov/nchs/products/databriefs/db285.htm) which focus a lot on how the deaths are distributed among different demographic groups; something important to notice is that they "standardize" some of the results as "deaths per 1000 live births" which must mean that they combined knowledge that we can distil from combining the linked data with the natality data. However, there are many other measured factors to consider. 

**11)** How do the different factors in the linked data differ in distribution from the natality data itself? Could you create an objective "industry" or target audience in which understanding such differences could have value?

In [13]:
import pandas as pd
import numpy as np
import sklearn

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Combining 3 of the 12 datasets from the large file to get 25% of the data

In [3]:
linked_bd = pd.read_csv('../../data/Linked_BD_17/Linked_2017_1.csv')

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [19]:
# linked_bd_2 = pd.read_csv('../../data/Linked_BD_17/Linked_2017_2.csv')

In [98]:
# linked_bd_3 = pd.read_csv('../../data/Linked_Bd_17/Linked_2017_3.csv')

In [55]:
# columns to get rid of if not already gone
def linked_cleaner(df):
    del df['birth_day_of_wk'] 
    del df['age_at_death_in_days'] 
    del df['age_at_death_intervals']
    del df['manner_of_death'] 
    del df['place_of_injury']
    del df['infant_cause_of_death']
    del df['mother_transferred']
    del df['mo_since_last_other_birth']
    
    return df

In [97]:
linked_bd = linked_cleaner(linked_bd)
# linked_bd_2 = linked_cleaner(linked_bd_2)
# linked_bd_3 = linked_cleaner(linked_bd_3)

In [33]:
# dfs = [linked_bd, linked_bd_2, linked_bd_3]
# linked_data = pd.concat(dfs)
# linked_data.shape # 966,196 rows, quarter of the entire 2017 period linked birth/death data set

,birth_year,birth_month,birth_place,mothers_age,mothers_age_recode,mothers_nativity,mothers_race,mothers_hispanic_origin,paternity_acknow,mothers_marital_status,mothers_education,fathers_age,fathers_race,fathers_hispanic_origin,fathers_education,prior_living_births,prior_dead_births,prior_terminations,mo_since_last_live_birth,mo_prenatal_care_began,n_prenatal_visits,wic,cigs_before_preg,cigs_tri1,cigs_tri2,cigs_tri3,mothers_bmi,pre_preg_lbs,weight_gain,pre_preg_diab,gest_diab,pre_preg_hypten,gest_hypten,hypten_ecl,prev_preterm_birth,infertility_treatment,fertil_enhance,asst_repro_tech,prev_cesar,no_risk_reported,gonorrhea,syphilis,chlamydia,hepB,hepC,no_infection_reported,success_ext_cep,fail_ext_cep,induced_labor,aug_labor,steriods,antibiotics,chorioamnionitis,anesthesia,fetal_present_at_birth,trial_of_labor_attempt,delivery_method,maternal_transfusion,perineal_laceration,rupt_uterus,unplanned_hyster,admit_to_ICU,attendant_at_birth,payment_source,APGAR_score_5min,APGAR_score_10min,sex_of_infant,obst_est_of_gestation_used,combined_gestation_week,combined_gestation_week_recode,birth_weight_gm,birth_weight_gm_recode,assist_vent_immed,assist_vent_after6,admit_NICU,surfactant,antibiotics_for_newborn,seizures,anencephaly,meningo_spina_bif,cyn_cong_heart_disease,cong_diaph_hernia,omphalocele,gastroschisis,limb_reduc_defect,cleft_lip_or_palate,cleft_palate_only,down_syndr,suspect_chromo_disorder,hypospadias,no_cong_anamolies_checked,infant_living_at_report,infant_breastfed_at_discharge
0,2017,1,1,31,5,1,2,0,X,1,5,5,2,0,4,3,0,0,5,1,11,Y,0,0,0,0,4,220,16,N,N,N,N,N,N,N,X,X,N,1,N,N,N,N,N,1,N,N,N,N,N,N,N,Y,1,X,1,N,N,N,N,N,3,4,9,88,F,,40,8,,10,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,1,Y,Y
1,2017,1,1,26,4,1,1,0,Y,2,3,4,1,0,4,2,0,0,2,2,10,N,0,0,0,0,2,113,25,N,N,N,N,N,Y,N,X,X,N,0,N,N,N,N,N,1,N,N,N,N,N,Y,N,N,1,X,1,N,N,N,N,N,1,1,9,88,M,,39,7,,9,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,1,Y,Y
2,2017,1,2,35,6,1,1,0,X,1,7,6,1,0,4,2,0,0,5,1,12,N,0,0,0,0,2,170,32,N,N,N,N,N,N,N,X,X,N,1,N,N,N,N,N,1,N,N,N,N,N,N,N,N,1,X,1,N,N,N,N,N,3,2,9,88,F,,38,6,,9,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,1,Y,Y
3,2017,1,1,26,4,1,1,0,N,2,3,11,9,9,9,1,0,3,7,2,22,Y,20,10,5,5,2,125,48,N,N,N,N,N,N,N,X,X,N,1,N,N,N,N,N,1,N,N,Y,N,N,Y,N,N,1,X,1,N,N,N,N,N,1,1,9,88,F,,41,9,,8,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,1,Y,Y
4,2017,1,1,38,6,1,1,0,X,1,5,5,1,0,4,1,0,0,4,1,11,N,0,0,0,0,3,164,36,N,N,N,N,N,N,N,X,X,N,1,N,N,N,N,N,1,N,N,N,N,N,N,N,N,1,X,1,N,N,N,N,N,1,2,9,88,M,,41,9,,10,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,1,Y,Y


In [ ]:
# linked_data.to_csv('mortality_2017.csv')

# Grouped By Mother's Race, Mother's Education, Admittance to NICU

Mother's Race
- 1 = White (only)
- 2 = Black (only)
- 3 = American Indian Alaskan Native (only)
- 4 = Asian (only)
- 5 = Native Hawaiians and Pacific Islanders (only)
- 6 = More than one race (mixed)

Mother's Education
- 1 = 8th grade or less
- 2 = some high school no diploma
- 3 = high school grad
- 4 = some college no degree
- 5 = Associates
- 6 = Bachelors
- 7 = Masters
- 8 = Doctorate
- 9 = Unknown

In [ ]:
# DO NOT RUN THIS CELL; helper function for getting the mode of a groupedby column
#.agg(lambda x: x.value_counts().index[0])

In [94]:
# total counts of babies admitted to NICU, omitting babies with Unknown admittance
df_race_edu_nicu = linked_bd[linked_bd.admit_NICU != 'U'].groupby(['mothers_race', 'mothers_education', 'admit_NICU']).agg({'birth_year':'count'})
df_race_edu_nicu.rename(columns={'birth_year': 'total_deaths'}, inplace=True)
df_race_edu_nicu

total_deaths
mothers_race mothers_education admit_NICU              
1            1                 N                   7469
                               Y                    632
             2                 N                  19960
                               Y                   1995
             3                 N                  51767
                               Y                   4966
             4                 N                  42398
                               Y                   4055
             5                 N                  18362
                               Y                   1716
             6                 N                  46649
                               Y                   3787
             7                 N                  20352
                               Y                   1761
             8                 N                   5796
                               Y                    456
             9                 N                   2440
                               Y                    330
2            1                 N                   1223
                               Y                    133
             2                 N                   5828
                               Y                    812
             3                 N                  15761
                               Y                   2119
             4                 N                  11777
                               Y                   1545
             5                 N                   3347
                               Y                    447
             6                 N                   5132
                               Y                    619
             7                 N                   2069
                               Y                    251
             8                 N                    463
                               Y                     57
             9                 N                    483
                               Y                     83
3            1                 N                     65
                               Y                     11
             2                 N                    572
                               Y                     73
             3                 N                    964
                               Y                    114
             4                 N                    638
                               Y                     60
             5                 N                    194
                               Y                     22
             6                 N                    154
                               Y                     17
             7                 N                     40
                               Y                     10
             8                 N                     10
             9                 N                     25
                               Y                      7
4            1                 N                    661
                               Y                     61
             2                 N                   1055
                               Y                     94
             3                 N                   2590
                               Y                    244
             4                 N                   2183
                               Y                    205
             5                 N                   1300
                               Y                    113
             6                 N                   6665
                               Y                    571
             7                 N                   4041
                               Y                    364
             8                 N                   1516
                               Y                     97
             9                 N                    568
           

In [95]:
# percentage of dead babies admitted to NICU grouped by mother's race and mother's education
df_race_edu_nicu['percent_deaths'] = df_race_edu_nicu*100 / df_race_edu_nicu.groupby(level=['mothers_race', 'mothers_education']).sum()
df_race_edu_nicu['percent_deaths'] = df_race_edu_nicu['percent_deaths'].round(1)
df_race_edu_nicu

total_deaths  percent_deaths
mothers_race mothers_education admit_NICU                              
1            1                 N                   7469            92.2
                               Y                    632             7.8
             2                 N                  19960            90.9
                               Y                   1995             9.1
             3                 N                  51767            91.2
                               Y                   4966             8.8
             4                 N                  42398            91.3
                               Y                   4055             8.7
             5                 N                  18362            91.5
                               Y                   1716             8.5
             6                 N                  46649            92.5
                               Y                   3787             7.5
             7                 N                  20352            92.0
                               Y                   1761             8.0
             8                 N                   5796            92.7
                               Y                    456             7.3
             9                 N                   2440            88.1
                               Y                    330            11.9
2            1                 N                   1223            90.2
                               Y                    133             9.8
             2                 N                   5828            87.8
                               Y                    812            12.2
             3                 N                  15761            88.1
                               Y                   2119            11.9
             4                 N                  11777            88.4
                               Y                   1545            11.6
             5                 N                   3347            88.2
                               Y                    447            11.8
             6                 N                   5132            89.2
                               Y                    619            10.8
             7                 N                   2069            89.2
                               Y                    251            10.8
             8                 N                    463            89.0
                               Y                     57            11.0
             9                 N                    483            85.3
                               Y                     83            14.7
3            1                 N                     65            85.5
                               Y                     11            14.5
             2                 N                    572            88.7
                               Y                     73            11.3
             3                 N                    964            89.4
                               Y                    114            10.6
             4                 N                    638            91.4
                               Y                     60             8.6
             5                 N                    194            89.8
                               Y                     22            10.2
             6                 N                    154            90.1
                               Y                     17             9.9
             7                 N                     40            80.0
                               Y                     10            20.0
             8                 N                     10           100.0
             9                 N                     25            78.1
                               Y                      7            21.9
4            1                 N                    661            91.6
           

# More simplified version looking at
# ONLY MOTHER'S RACE and NICU ADMITTANCE

In [89]:
# create dataframe based on 
df_race_nicu = linked_bd[linked_bd.admit_NICU != 'U'].groupby(['mothers_race', 'admit_NICU']).agg({'birth_year':'count'})
df_race_nicu.rename(columns={'birth_year': 'total_deaths'}, inplace=True)
df_race_nicu

total_deaths
mothers_race admit_NICU              
1            N                 215193
             Y                  19698
2            N                  46083
             Y                   6066
3            N                   2662
             Y                    314
4            N                  20579
             Y                   1804
5            N                    893
             Y                     73
6            N                   7647
             Y                    780

In [91]:
# create new column for corresponding percent deaths based on MOTHER'S RACE and NICU ADMITTANCE
df_race_nicu['percent_deaths'] = df_race_nicu*100 / df_race_nicu.groupby(level='mothers_race').sum()
df_race_nicu['percent_deaths'] = df_race_nicu['percent_deaths'].round(1)
df_race_nicu

total_deaths  percent_deaths
mothers_race admit_NICU                              
1            N                 215193            91.6
             Y                  19698             8.4
2            N                  46083            88.4
             Y                   6066            11.6
3            N                   2662            89.4
             Y                    314            10.6
4            N                  20579            91.9
             Y                   1804             8.1
5            N                    893            92.4
             Y                     73             7.6
6            N                   7647            90.7
             Y                    780             9.3

# Grouped By Mother's Age (Recode), Mother's Education

Mother's Age (Recode)
- 1 = Under 15 years
- 2 = 15 to 19 years
- 3 = 20 to 24 years
- 4 = 25 to 29 years
- 5 = 30 to 34 years
- 6 = 35 to 39 years
- 7 = 40 to 44 years
- 8 = 45 to 49 years
- 9 = 50 to 54 years

Mother's Education
- 1 = 8th grade or less
- 2 = some high school no diploma
- 3 = high school grad
- 4 = some college no degree
- 5 = Associates
- 6 = Bachelors
- 7 = Masters
- 8 = Doctorate
- 9 = Unknown

In [85]:
# dataframe with total deaths based on mother's age and education
df_age_edu = linked_bd.groupby(['mothers_age_recode', 'mothers_education']).agg({'birth_year': 'count'})
df_age_edu.rename(columns={'birth_year': 'total_deaths'}, inplace=True)
df_age_edu

total_deaths
mothers_age_recode mothers_education              
1                  1                           106
                   2                            60
                   9                             3
2                  1                           594
                   2                          6987
                   3                          6675
                   4                          1664
                   5                            69
                   6                             5
                   7                             1
                   8                             1
                   9                           136
3                  1                          1699
                   2                          8782
                   3                         27827
                   4                         17771
                   5                          3603
                   6                          3151
                   7                           236
                   8                            22
                   9                           592
4                  1                          2540
                   2                          7560
                   3                         24263
                   4                         22657
                   5                          9466
                   6                         19668
                   7                          5709
                   8                          1020
                   9                          1024
5                  1                          2700
                   2                          4887
                   3                         14111
                   4                         15077
                   5                          8487
                   6                         26714
                   7                         13842
                   8                          3979
                   9                          1280
6                  1                          2053
                   2                          2592
                   3                          6682
                   4                          6816
                   5                          3836
                   6                         12888
                   7                          8018
                   8                          2904
                   9                           783
7                  1                           654
                   2                           652
                   3                          1472
                   4                          1404
                   5                           773
                   6                          2396
                   7                          1501
                   8                           586
                   9                           237
8                  1                            57
                   2                            33
                   3                           112
                   4                            76
                   5                            46
                   6                           203
                   7                           132
                   8                            65
                   9                            40
9                  1                             4
                   2                             2
                   3                             8
                   4                             8
                   5                             5
                   6                            14
                   7                            13
                   8                            13
                   9                            20

In [86]:
# make new column for corresponding death percentages based on mother's age and education
df_age_edu['percent_deaths'] = df_age_edu*100 / df_age_edu.groupby(level='mothers_age_recode').sum()
df_age_edu['percent_deaths'] = df_age_edu['percent_deaths'].round(1)
df_age_edu

total_deaths  percent_deaths
mothers_age_recode mothers_education                              
1                  1                           106            62.7
                   2                            60            35.5
                   9                             3             1.8
2                  1                           594             3.7
                   2                          6987            43.3
                   3                          6675            41.4
                   4                          1664            10.3
                   5                            69             0.4
                   6                             5             0.0
                   7                             1             0.0
                   8                             1             0.0
                   9                           136             0.8
3                  1                          1699             2.7
                   2                          8782            13.8
                   3                         27827            43.7
                   4                         17771            27.9
                   5                          3603             5.7
                   6                          3151             4.9
                   7                           236             0.4
                   8                            22             0.0
                   9                           592             0.9
4                  1                          2540             2.7
                   2                          7560             8.1
                   3                         24263            25.8
                   4                         22657            24.1
                   5                          9466            10.1
                   6                         19668            20.9
                   7                          5709             6.1
                   8                          1020             1.1
                   9                          1024             1.1
5                  1                          2700             3.0
                   2                          4887             5.4
                   3                         14111            15.5
                   4                         15077            16.6
                   5                          8487             9.3
                   6                         26714            29.3
                   7                         13842            15.2
                   8                          3979             4.4
                   9                          1280             1.4
6                  1                          2053             4.4
                   2                          2592             5.6
                   3                          6682            14.3
                   4                          6816            14.6
                   5                          3836             8.2
                   6                         12888            27.7
                   7                          8018            17.2
                   8                          2904             6.2
                   9                           783             1.7
7                  1                           654             6.8
                   2                           652             6.7
                   3                          1472            15.2
                   4                          1404            14.5
                   5                           773             8.0
                   6                          2396            24.8
                   7                          1501            15.5
                   8                           586             6.1
                   9                           237             2.4
8                  1                            57             7.5
                  

In [88]:
# filter out any moms/babies with death percentage greater than 0
df_age_edu = df_age_edu[df_age_edu.percent_deaths > 0.0]
df_age_edu

total_deaths  percent_deaths
mothers_age_recode mothers_education                              
1                  1                           106            62.7
                   2                            60            35.5
                   9                             3             1.8
2                  1                           594             3.7
                   2                          6987            43.3
                   3                          6675            41.4
                   4                          1664            10.3
                   5                            69             0.4
                   9                           136             0.8
3                  1                          1699             2.7
                   2                          8782            13.8
                   3                         27827            43.7
                   4                         17771            27.9
                   5                          3603             5.7
                   6                          3151             4.9
                   7                           236             0.4
                   9                           592             0.9
4                  1                          2540             2.7
                   2                          7560             8.1
                   3                         24263            25.8
                   4                         22657            24.1
                   5                          9466            10.1
                   6                         19668            20.9
                   7                          5709             6.1
                   8                          1020             1.1
                   9                          1024             1.1
5                  1                          2700             3.0
                   2                          4887             5.4
                   3                         14111            15.5
                   4                         15077            16.6
                   5                          8487             9.3
                   6                         26714            29.3
                   7                         13842            15.2
                   8                          3979             4.4
                   9                          1280             1.4
6                  1                          2053             4.4
                   2                          2592             5.6
                   3                          6682            14.3
                   4                          6816            14.6
                   5                          3836             8.2
                   6                         12888            27.7
                   7                          8018            17.2
                   8                          2904             6.2
                   9                           783             1.7
7                  1                           654             6.8
                   2                           652             6.7
                   3                          1472            15.2
                   4                          1404            14.5
                   5                           773             8.0
                   6                          2396            24.8
                   7                          1501            15.5
                   8                           586             6.1
                   9                           237             2.4
8                  1                            57             7.5
                   2                            33             4.3
                   3                           112            14.7
                   4                            76             9.9
                   5                            46             6.0
                  